# Automatic Prompt Engineer (APE) Example

This notebook demonstrates **Automatic Prompt Engineer (APE)**, a meta-learning technique where the AI writes its own prompts.

**The Concept:**
Writing the "perfect" prompt is hard. APE automates this by:
1.  **Generation**: The AI writes multiple different system prompts for a specific task.
2.  **Evaluation**: It tests each prompt against a small set of "Ground Truth" examples (Input + Correct Answer).
3.  **Selection**: It calculates an accuracy score for each prompt and selects the best one.

**Key Mechanics:**
* **The Architect**: A model that designs prompts.
* **The Judge**: A scoring loop that runs candidates against test cases.
* **The Result**: A scientifically optimized prompt that outperforms manual guessing.

In [ ]:
%pip install openai python-dotenv --quiet

### 1. Setup and Authorization

We start by importing the necessary libraries and loading your OpenAI API key.

In [ ]:
from openai import OpenAI
import os
import random
from dotenv import load_dotenv

load_dotenv()

api_key = os.getenv("OPENAI_API_KEY")
if not api_key:
    api_key = input("Paste your OpenAI API key: ").strip()

client = OpenAI(api_key=api_key)
print("OpenAI client ready!")

### 2. The Architect: Generating Candidate Prompts

In this step, we ask the model (the "Architect") to brainstorm 5–8 distinct, high-quality system prompts for a task you define. We use a high temperature to ensure diversity in the wording and strategies.

In [ ]:
def ape_generate_prompts(task, num_prompts=5):
    """Generates diverse system prompts for a specific task."""
    print(f"Architect is designing {num_prompts} prompts for: '{task}'...")
    
    try:
        response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[{
                "role": "user", 
                "content": f"""
Generate {num_prompts} diverse, creative, high-quality system prompts that would make an LLM excel at this task:

Task: {task}

Return ONLY a numbered list of the prompts. Do not include explanations.
"""
            }],
            temperature=1.0  # High creativity to get different angles
        )
        
        # Parse the output into a clean list
        raw_text = response.choices[0].message.content
        prompts = [
            line.strip()[line.strip().find(" ")+1:] 
            for line in raw_text.split("\n") 
            if line.strip() and any(c.isdigit() for c in line)
        ]
        return prompts
        
    except Exception as e:
        print(f"Error generating prompts: {e}")
        return []

### 3. The Judge: Evaluating Performance

This function is the "Test Harness." It takes a candidate prompt and runs it against your provided examples.

* If the model's output matches your expected output, the prompt gets a point.
* The final score is the percentage of correct answers (0.0 to 1.0).

In [ ]:
def ape_evaluate(prompts, test_examples):
    """
    Tests each prompt against a list of (Input, Expected_Output) tuples.
    Returns a sorted list of (Prompt, Accuracy_Score).
    """
    print("\nStarting evaluation tournament...")
    scores = []
    
    for i, prompt in enumerate(prompts):
        correct_count = 0
        print(f"  Testing Prompt #{i+1}...", end="\r")
        
        for input_text, expected_output in test_examples:
            try:
                # Run the candidate prompt
                response = client.chat.completions.create(
                    model="gpt-4o-mini",
                    messages=[
                        {"role": "system", "content": prompt},
                        {"role": "user", "content": input_text}
                    ],
                    temperature=0  # Zero temp for consistent testing
                )
                prediction = response.choices[0].message.content.strip()
                
                # Simple exact match or "contains" check
                if expected_output.lower() in prediction.lower():
                    correct_count += 1
                    
            except Exception as e:
                print(f"\n  Error testing prompt {i+1}: {e}")
                continue
        
        accuracy = correct_count / len(test_examples) if test_examples else 0
        scores.append((prompt, accuracy))
        
    print(f"  Evaluation complete!              ")
    # Sort by highest score
    return sorted(scores, key=lambda x: x[1], reverse=True)

### 4. The Optimization Loop

Now we put it all together.

1.  **Define Task**: You describe what you want (e.g., "Classify movie reviews").
2.  **Provide Data**: You give 3-4 examples of correct behavior.
3.  **Run APE**: The system writes prompts, tests them, and hands you the winner.

In [ ]:
# 1. Define the Task
task_description = input("Enter the task (e.g., 'Classify text as Spam or Not Spam'): ")

# 2. Collect Ground Truth Data
examples = []
print("\nNow provide 3-5 examples for testing.")
print("Type 'done' when finished.\n")

while True:
    inp = input(f"Example {len(examples)+1} Input: ")
    if inp.lower() == "done": break
    out = input(f"Example {len(examples)+1} Expected Output: ")
    examples.append((inp, out))

if not examples:
    print("No examples provided. Using dummy data for demo...")
    task_description = "Classify sentiment as Positive or Negative"
    examples = [
        ("I love this!", "Positive"),
        ("This is terrible.", "Negative"),
        ("Best day ever.", "Positive"),
        ("I want a refund.", "Negative")
    ]

# 3. Generate Candidates
candidates = ape_generate_prompts(task_description)

# 4. Evaluate & Rank
ranked_prompts = ape_evaluate(candidates, examples)

# 5. Show Results
print("\n" + "="*60)
print(f"🏆 WINNING PROMPT (Accuracy: {ranked_prompts[0][1]:.0%})")
print("="*60)
print(f"\"{ranked_prompts[0][0]}\"")
print("="*60)

print("\nRunner Up:")
if len(ranked_prompts) > 1:
    print(f"\"{ranked_prompts[1][0]}\" (Accuracy: {ranked_prompts[1][1]:.0%})")